In [1]:
import pandas as pd

In [4]:
from keras.preprocessing import sequence
from keras.layers import Embedding, Dense, Flatten
from keras.models import Sequential


In [6]:
from sklearn import preprocessing


In [2]:
df = pd.read_csv('../base/word2vec_corpus.csv', index_col=0)


In [3]:
df_cbow = pd.read_csv('../output/word2vec_cbow.csv', index_col=0)

In [7]:
le = preprocessing.LabelEncoder()

In [8]:
le.fit(df['sentimento'])
df['sentimento'] = le.transform(df['sentimento'])

In [13]:
from sklearn.model_selection import train_test_split


In [34]:
target = df['sentimento']
features = df.iloc[:,0:50]

In [35]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


In [39]:
#Dependencies
import keras
from keras.models import Sequential
from keras.layers import Dense
# Neural network
model = Sequential()
model.add(Dense(70, input_dim=50, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(1, activation='softmax'))


In [40]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [41]:
history = model.fit(X_train, y_train, epochs=100, batch_size=64)


Epoch 1/100
80/80 [==============================] - 1s 3ms/step - loss: 0.2364 - accuracy: 0.4376
Epoch 2/100
80/80 [==============================] - 0s 2ms/step - loss: 0.0512 - accuracy: 0.4376
Epoch 3/100
80/80 [==============================] - 0s 3ms/step - loss: -0.7372 - accuracy: 0.4376
Epoch 4/100
80/80 [==============================] - 0s 3ms/step - loss: -5.6823 - accuracy: 0.4376
Epoch 5/100
80/80 [==============================] - 0s 2ms/step - loss: -27.0660 - accuracy: 0.4376
Epoch 6/100
80/80 [==============================] - 0s 3ms/step - loss: -88.9697 - accuracy: 0.4376
Epoch 7/100
80/80 [==============================] - 0s 3ms/step - loss: -224.1871 - accuracy: 0.4376
Epoch 8/100
80/80 [==============================] - 0s 2ms/step - loss: -456.3086 - accuracy: 0.4376
Epoch 9/100
80/80 [==============================] - 0s 2ms/step - loss: -834.4924 - accuracy: 0.4376
Epoch 10/100
80/80 [==============================] - 0s 2ms/step - loss: -1404.3218 - accurac

80/80 [==============================] - 0s 2ms/step - loss: -3244990.2500 - accuracy: 0.4376
Epoch 79/100
80/80 [==============================] - 0s 2ms/step - loss: -3383273.0000 - accuracy: 0.4376
Epoch 80/100
80/80 [==============================] - 0s 3ms/step - loss: -3526252.0000 - accuracy: 0.4376
Epoch 81/100
80/80 [==============================] - 0s 3ms/step - loss: -3673338.2500 - accuracy: 0.4376
Epoch 82/100
80/80 [==============================] - 0s 3ms/step - loss: -3827493.7500 - accuracy: 0.4376
Epoch 83/100
80/80 [==============================] - 0s 2ms/step - loss: -3978445.7500 - accuracy: 0.4376
Epoch 84/100
80/80 [==============================] - 0s 3ms/step - loss: -4135045.7500 - accuracy: 0.4376
Epoch 85/100
80/80 [==============================] - 0s 3ms/step - loss: -4302140.0000 - accuracy: 0.4376
Epoch 86/100
80/80 [==============================] - 0s 3ms/step - loss: -4467703.0000 - accuracy: 0.4376
Epoch 87/100
80/80 [==============================

In [23]:
y_pred = model.predict(X_test)


40/40 [==============================] - 0s 2ms/step
